# Implementing Recommender Systems - Lab

## Introduction

In this lab, you'll practice creating a recommender system model using `surprise`. You'll also get the chance to create a more complete recommender system pipeline to obtain the top recommendations for a specific user.


## Objectives

In this lab you will: 

- Use surprise's built-in reader class to process data to work with recommender algorithms 
- Obtain a prediction for a specific user for a particular item 
- Introduce a new user with rating to a rating matrix and make recommendations for them 
- Create a function that will return the top n recommendations for a user 


For this lab, we will be using the famous 1M movie dataset. It contains a collection of user ratings for many different movies. In the last lesson, you were exposed to working with `surprise` datasets. In this lab, you will also go through the process of reading in a dataset into the `surprise` dataset format. To begin with, load the dataset into a Pandas DataFrame. Determine which columns are necessary for your recommendation system and drop any extraneous ones.

In [1]:
import pandas as pd
df = pd.read_csv('./ml-latest-small/ratings.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [4]:
# Drop unnecessary columns
new_df = df.drop('timestamp', axis=1)

It's now time to transform the dataset into something compatible with `surprise`. In order to do this, you're going to need `Reader` and `Dataset` classes. There's a method in `Dataset` specifically for loading dataframes.

In [5]:
from surprise import Reader, Dataset
# read in values as Surprise dataset 
reader = Reader()
data = Dataset.load_from_df(new_df,reader)

Let's look at how many users and items we have in our dataset. If using neighborhood-based methods, this will help us determine whether or not we should perform user-user or item-item similarity

In [6]:
dataset = data.build_full_trainset()
print('Number of users: ', dataset.n_users, '\n')
print('Number of items: ', dataset.n_items)

Number of users:  610 

Number of items:  9724


## Determine the best model 

Now, compare the different models and see which ones perform best. For consistency sake, use RMSE to evaluate models. Remember to cross-validate! Can you get a model with a higher average RMSE on test data than 0.869?

In [7]:
# importing relevant libraries
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import SVD
from surprise.prediction_algorithms import KNNWithMeans, KNNBasic, KNNBaseline
from surprise.model_selection import GridSearchCV
import numpy as np

In [41]:
## Perform a gridsearch with SVD
# ⏰ This cell may take several minutes to run

params = {'n_factors':[42], 'reg_all':[.032]}

gs_svd = GridSearchCV(SVD, param_grid=params, n_jobs=-1)

gs_svd.fit(data)

In [42]:
# print out optimal parameters for SVD after GridSearch
print(gs_svd.best_score)
print(gs_svd.best_params)

{'rmse': 0.8701687699418498, 'mae': 0.6689075861173378}
{'rmse': {'n_factors': 42, 'reg_all': 0.032}, 'mae': {'n_factors': 42, 'reg_all': 0.032}}


In [43]:
# cross validating with KNNBasic
knn_basic = KNNBasic(sim_options={'name':'pearson', 'user_based':True})
cv_knn_basic = cross_validate(knn_basic, data, n_jobs=-1)

In [44]:
# print out the average RMSE score for the test set
for i in cv_knn_basic.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_basic['test_rmse']))

('test_rmse', array([0.96806542, 0.97761268, 0.97478684, 0.97738971, 0.971186  ]))
('test_mae', array([0.74783905, 0.75506036, 0.75207555, 0.75480236, 0.7498266 ]))
('fit_time', (1.417170763015747, 1.4287960529327393, 1.5581181049346924, 1.5438153743743896, 1.4579355716705322))
('test_time', (2.0831894874572754, 1.792677402496338, 1.922363042831421, 1.8742244243621826, 2.0523993968963623))
-----------------------
0.9738081292831474


In [45]:
# cross validating with KNNBaseline
knn_baseline = KNNBaseline(sim_options={'name':'pearson', 'user_based':True})
cv_knn_baseline = cross_validate(knn_baseline,data)

Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.


In [46]:
# print out the average score for the test set
for i in cv_knn_baseline.items():
    print(i)

np.mean(cv_knn_baseline['test_rmse'])

('test_rmse', array([0.87793532, 0.88150181, 0.88126258, 0.87275159, 0.8739568 ]))
('test_mae', array([0.67019854, 0.67142083, 0.67454359, 0.66707992, 0.66722188]))
('fit_time', (0.7130053043365479, 0.7422237396240234, 0.7799108028411865, 0.8004333972930908, 0.7237777709960938))
('test_time', (2.212355613708496, 2.2387776374816895, 2.2913856506347656, 2.3185558319091797, 2.3023905754089355))


0.8774816207756613

Based off these outputs, it seems like the best performing model is the SVD model with `n_factors = 50` and a regularization rate of 0.05. Use that model or if you found one that performs better, feel free to use that to make some predictions.

## Making Recommendations

It's important that the output for the recommendation is interpretable to people. Rather than returning the `movie_id` values, it would be far more valuable to return the actual title of the movie. As a first step, let's read in the movies to a dataframe and take a peek at what information we have about them.

In [47]:
df_movies = pd.read_csv('./ml-latest-small/movies.csv')

In [48]:
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


## Making simple predictions
Just as a reminder, let's look at how you make a prediction for an individual user and item. First, we'll fit the SVD model we had from before.

In [49]:
svd = SVD(n_factors= 42, reg_all=0.032)
svd.fit(dataset)

In [50]:
svd.predict(2, 4)

Prediction(uid=2, iid=4, r_ui=None, est=3.1448333092973613, details={'was_impossible': False})

This prediction value is a tuple and each of the values within it can be accessed by way of indexing. Now let's put our knowledge of recommendation systems to do something interesting: making predictions for a new user!

## Obtaining User Ratings 

It's great that we have working models and everything, but wouldn't it be nice to get to recommendations specifically tailored to your preferences? That's what we'll be doing now. The first step is to create a function that allows us to pick randomly selected movies. The function should present users with a movie and ask them to rate it. If they have not seen the movie, they should be able to skip rating it. 

The function `movie_rater()` should take as parameters: 

* `movie_df`: DataFrame - a dataframe containing the movie ids, name of movie, and genres
* `num`: int - number of ratings
* `genre`: string - a specific genre from which to draw movies

The function returns:
* rating_list : list - a collection of dictionaries in the format of {'userId': int , 'movieId': int , 'rating': float}

#### This function is optional, but fun :) 

In [62]:
def movie_rater(movie_df,num, genre=None):
    userId = 666
    ratings = []
    if genre:
        movies = movie_df[movie_df['genres'].str.contains(genre)]
    else:
        movies = movie_df
    while num > 0:
        movie = movies.sample(1)
        print(movie)
        rating = input("If you have seen this movie, enter rating (1-5):\n(or press 'n' if not):")
        if rating == 'n':
            continue
        else:
            ratings.append({'userId':userId,'movieId':movie['movieId'].values[0],'rating':rating})
            num -= 1
    return ratings

If you're struggling to come up with the above function, you can use this list of user ratings to complete the next segment

In [67]:
user_rating = movie_rater(df_movies, 5, genre='Comedy')

     movieId                           title         genres
344      387  Low Down Dirty Shame, A (1994)  Action|Comedy
If you have seen this movie, enter rating (1-5):
(or press 'n' if not):n
     movieId                   title                  genres
643      829  Joe's Apartment (1996)  Comedy|Fantasy|Musical
If you have seen this movie, enter rating (1-5):
(or press 'n' if not):2
      movieId                    title          genres
4284     6254  Awful Truth, The (1937)  Comedy|Romance
If you have seen this movie, enter rating (1-5):
(or press 'n' if not):n
      movieId                                        title        genres
5764    31162  Life and Death of Peter Sellers, The (2004)  Comedy|Drama
If you have seen this movie, enter rating (1-5):
(or press 'n' if not):n
      movieId            title                genres
8037    98230  10 Years (2011)  Comedy|Drama|Romance
If you have seen this movie, enter rating (1-5):
(or press 'n' if not):n
      movieId                  

In [68]:
user_rating

[{'userId': 666, 'movieId': 829, 'rating': '2'},
 {'userId': 666, 'movieId': 317, 'rating': '4'},
 {'userId': 666, 'movieId': 2907, 'rating': '3'},
 {'userId': 666, 'movieId': 5218, 'rating': '2'},
 {'userId': 666, 'movieId': 2145, 'rating': '4'}]

### Making Predictions With the New Ratings
Now that you have new ratings, you can use them to make predictions for this new user. The proper way this should work is:

* add the new ratings to the original ratings DataFrame, read into a `surprise` dataset 
* train a model using the new combined DataFrame
* make predictions for the user
* order those predictions from highest rated to lowest rated
* return the top n recommendations with the text of the actual movie (rather than just the index number) 

In [70]:
## add the new ratings to the original ratings DataFrame
new_ratings = new_df.append(user_rating, ignore_index=True)
new_data = Dataset.load_from_df(new_ratings,reader)

In [71]:
# train a model using the new combined DataFrame
svd_new = SVD(n_factors=42, reg_all=.032)
svd_new.fit(new_data.build_full_trainset())

In [90]:
# make predictions for the user
# you'll probably want to create a list of tuples in the format (movie_id, predicted_score)
all_movies = []
for id in new_df['movieId'].unique():
    all_movies.append( (id,svd_new.predict(1000,id)[3]))

In [91]:
# order the predictions from highest to lowest rated

ranked_movies = sorted(all_movies, key=lambda x:x[1], reverse=True)

In [92]:
ranked_movies

[(318, 4.398026978219455),
 (1204, 4.377149883798303),
 (750, 4.319740346821389),
 (904, 4.301363471962797),
 (1104, 4.300312901188712),
 (898, 4.293522875202239),
 (2959, 4.286408491379461),
 (858, 4.277439457832962),
 (50, 4.251971650275778),
 (1276, 4.241788846939149),
 (3451, 4.2414880092251535),
 (1213, 4.240288594071153),
 (741, 4.238439716153224),
 (48516, 4.236800370589908),
 (3275, 4.234856855779973),
 (912, 4.2273412381826505),
 (1221, 4.226812466679872),
 (296, 4.2263732766936055),
 (1198, 4.224543643268712),
 (1178, 4.222817272681382),
 (2324, 4.219200674189682),
 (1197, 4.218394326815552),
 (246, 4.217374819786485),
 (7361, 4.215515950864965),
 (922, 4.215513944500317),
 (1089, 4.214375704435341),
 (2329, 4.212480973880414),
 (1199, 4.211404097804188),
 (260, 4.21020969120904),
 (1208, 4.210197990821413),
 (1248, 4.208878793231599),
 (2019, 4.206566455147206),
 (4973, 4.205079828700862),
 (1262, 4.20361975878391),
 (1217, 4.201513676207319),
 (1196, 4.1999619759611235),
 (

 For the final component of this challenge, it could be useful to create a function `recommended_movies()` that takes in the parameters:
* `user_ratings`: list - list of tuples formulated as (user_id, movie_id) (should be in order of best to worst for this individual)
* `movie_title_df`: DataFrame 
* `n`: int - number of recommended movies 

The function should use a `for` loop to print out each recommended *n* movies in order from best to worst

In [98]:
# return the top n recommendations using the 
def recommended_movies(ranked,movie_title_df,n):
    for i, rec in enumerate(ranked[:n]):
        title = movie_title_df[movie_title_df['movieId']==rec[0]]['title']
        print(f"Recommendation # {i+1}:\t{title}")

recommended_movies(ranked_movies,df_movies,5)

Recommendation # 1:	277    Shawshank Redemption, The (1994)
Name: title, dtype: object
Recommendation # 2:	906    Lawrence of Arabia (1962)
Name: title, dtype: object
Recommendation # 3:	602    Dr. Strangelove or: How I Learned to Stop Worr...
Name: title, dtype: object
Recommendation # 4:	686    Rear Window (1954)
Name: title, dtype: object
Recommendation # 5:	841    Streetcar Named Desire, A (1951)
Name: title, dtype: object


## Level Up (Optional)

* Try and chain all of the steps together into one function that asks users for ratings for a certain number of movies, then all of the above steps are performed to return the top $n$ recommendations
* Make a recommender system that only returns items that come from a specified genre

## Summary

In this lab, you got the chance to implement a collaborative filtering model as well as retrieve recommendations from that model. You also got the opportunity to add your own recommendations to the system to get new recommendations for yourself! Next, you will learn how to use Spark to make recommender systems.